In [53]:
import pybamm

In [54]:
# Use only the derived coefficients
# For Li+
coffe1_n = pybamm.FullBroadcast(
        pybamm.Scalar(3.75e-11), "negative elOxtrode", )
coffe1_s = pybamm.FullBroadcast(
        pybamm.Scalar(9.666472986565472e-11), "separator", )
coffe1_p = pybamm.FullBroadcast(
        pybamm.Scalar(5.816858043652089e-11), "positive elOxtrode", )
coffe1 = pybamm.concatenation(coffe1_n, coffe1_s, coffe1_p )
coffe2_n = pybamm.FullBroadcast(
        pybamm.Scalar(-3.1249999999999995e-11), "negative elOxtrode", )
coffe2_s = pybamm.FullBroadcast(
        pybamm.Scalar(-8.055394155471226e-11), "separator", )
coffe2_p = pybamm.FullBroadcast(
        pybamm.Scalar(-4.847381703043407e-11), "positive elOxtrode", )
coffe2 = pybamm.concatenation(coffe2_n, coffe2_s, coffe2_p )
coffe3_n = pybamm.FullBroadcast(
        pybamm.Scalar(1.9136951216600957), "negative elOxtrode", )
coffe3_s = pybamm.FullBroadcast(
        pybamm.Scalar(0), "separator", )
coffe3_p = pybamm.FullBroadcast(
        pybamm.Scalar(-2.3745495341343483), "positive elOxtrode", )
coffe3 = pybamm.concatenation(coffe3_n, coffe3_s, coffe3_p )
# For Ox
coffOx1_n = pybamm.FullBroadcast(
        pybamm.Scalar(-1.2499999999999998e-10), "negative elOxtrode", )
coffOx1_s = pybamm.FullBroadcast(
        pybamm.Scalar(-3.2221576621884903e-10), "separator", )
coffOx1_p = pybamm.FullBroadcast(
        pybamm.Scalar(-1.9389526812173628e-10), "positive elOxtrode", )
coffOx1 = pybamm.concatenation(coffOx1_n, coffOx1_s, coffOx1_p )
coffOx2_n = pybamm.FullBroadcast(
        pybamm.Scalar(6.25e-11), "negative elOxtrode", )
coffOx2_s = pybamm.FullBroadcast(
        pybamm.Scalar(1.6110788310942454e-10), "separator", )
coffOx2_p = pybamm.FullBroadcast(
        pybamm.Scalar(9.694763406086815e-11), "positive elOxtrode", )
coffOx2 = pybamm.concatenation(coffOx2_n, coffOx2_s, coffOx2_p )

coffOx3_n = pybamm.FullBroadcast(
        pybamm.Scalar(-4.045348278871794), "negative elOxtrode", )
coffOx3_s = pybamm.FullBroadcast(
        pybamm.Scalar(0), "separator", )
coffOx3_p = pybamm.FullBroadcast(
        pybamm.Scalar(0), "positive elOxtrode", )
coffOx3 = pybamm.concatenation(coffOx3_n, coffOx3_s, coffOx3_p )

In [55]:
model = pybamm.BaseModel()
c_e_n  = pybamm.Variable("Li+ concentration in negative elOxtrode", domain="negative elOxtrode")
c_Ox_n = pybamm.Variable("Ox concentration in negative elOxtrode", domain="negative elOxtrode")

c_e_s  = pybamm.Variable("Li+ concentration in separator", domain="separator")
c_Ox_s = pybamm.Variable("Ox concentration in separator", domain="separator")

c_e_p  = pybamm.Variable("Li+ concentration in positive elOxtrode", domain="positive elOxtrode")
c_Ox_p = pybamm.Variable("Ox concentration in positive elOxtrode", domain="positive elOxtrode")

c_Ox = pybamm.concatenation(c_Ox_n, c_Ox_s, c_Ox_p)
c_e  = pybamm.concatenation(c_e_n, c_e_s, c_e_p)
model.variables = {
    "Li+ concentration in negative elOxtrode":c_e_n,
    "Li+ concentration in separator":c_e_s,
    "Li+ concentration in positive elOxtrode":c_e_p,
    "Ox concentration in negative elOxtrode":c_Ox_n,
    "Ox concentration in separator":c_Ox_s,
    "Ox concentration in positive elOxtrode":c_Ox_p,
}

In [56]:
model.variables.update(
    {
        "Li+ concentration": c_e, 
        "Ox concentration": c_Ox,
    }
)

In [57]:
print(model.variables.keys())

dict_keys(['Li+ concentration in negative elOxtrode', 'Li+ concentration in separator', 'Li+ concentration in positive elOxtrode', 'Ox concentration in negative elOxtrode', 'Ox concentration in separator', 'Ox concentration in positive elOxtrode', 'Li+ concentration', 'Ox concentration'])


In [58]:
model.length_scales = {
    "negative elOxtrode": 8.52e-5,
    "separator": 1.2e-5,
    "positive elOxtrode": 7.56e-5,
}
D_e     = 3e-10;    # Li+ diffusivity in m2/s
model.timescale = pybamm.Scalar(8.52e-5 ** 2 /  D_e) 

In [59]:
model.rhs = {
    c_e: 
    coffe1  * pybamm.div(pybamm.grad(c_e))
    +coffe2 * pybamm.div(pybamm.grad(c_Ox))
    + coffe3,

    c_Ox:
    coffOx1   * pybamm.div(pybamm.grad(c_e))
    + coffOx2 * pybamm.div(pybamm.grad(c_Ox))
    + coffOx3,
} 

In [60]:
print(coffe1)

concatenation(broadcast(3.75e-11), broadcast(9.666472986565472e-11), broadcast(5.816858043652089e-11))


In [61]:
model.initial_conditions = {c_e: pybamm.Scalar(1000),c_Ox: pybamm.Scalar(4000)}

model.boundary_conditions = {
    c_e: {"left": (pybamm.Scalar(0), "Neumann"),"right": (pybamm.Scalar(0), "Neumann")},  
    c_Ox:{"left": (pybamm.Scalar(0), "Neumann"),"right": (pybamm.Scalar(0), "Neumann")} }
# define geometry
x_n = pybamm.SpatialVariable(
    "x_n", 
    domain=["negative elOxtrode"], 
    coord_sys="cartesian"
)
x_s = pybamm.SpatialVariable(
    "x_s", 
    domain=["separator"], 
    coord_sys="cartesian"
)
x_p = pybamm.SpatialVariable(
    "x_p", 
    domain=["positive elOxtrode"], 
    coord_sys="cartesian"
)
geometry = {
    "negative elOxtrode": {x_n: {"min": pybamm.Scalar(0), "max": pybamm.Scalar(8.52e-5)}},
    "separator":          {x_s: {"min": pybamm.Scalar(8.52e-5), "max": pybamm.Scalar(9.72e-5)}},
    "positive elOxtrode": {x_p: {"min": pybamm.Scalar(9.72e-5), "max": pybamm.Scalar(17.28e-5)}},
}
# mesh and discretise ,"separator","positive elOxtrode"
submesh_types = {
    "negative elOxtrode": pybamm.MeshGenerator(pybamm.Uniform1DSubMesh),
    "separator": pybamm.MeshGenerator(pybamm.Uniform1DSubMesh),
    "positive elOxtrode": pybamm.MeshGenerator(pybamm.Uniform1DSubMesh),
}
var_pts = {x_n: 30, x_s: 21, x_p: 30}
mesh = pybamm.Mesh(geometry, submesh_types, var_pts)
spatial_methods = {
    "negative elOxtrode": pybamm.FiniteVolume(),
    "separator": pybamm.FiniteVolume(),
    "positive elOxtrode": pybamm.FiniteVolume(),
}
disc = pybamm.Discretisation(mesh, spatial_methods)
disc.process_model(model);

In [62]:
# solve
solver = pybamm.ScipySolver()
solution = solver.solve(model, [0,1])


In [63]:
pybamm.dynamic_plot(
    solution,
    [
        "Li+ concentration",
        "Ox concentration",
    ],
)

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…